Parameter studies are organized in study cases, operation scenarios and variations in PowerFactory. This tutorials shows how to create parameter studies with ease and fully automated using the study cases interface of *powfacpy*.  

First, we activate the PowerFactory project as in the *getting started* tutorial.

In [ ]:
# If you use IPython/Jupyter:
import sys
sys.path.append(r'C:\Program Files\DIgSILENT\PowerFactory 2022 SP1\Python\3.10') # you may use a different directory
# Get the PF app
import powerfactory as powerfactory
app = powerfactory.GetApplication()
import powfacpy
app.Show()
app.ActivateProject(r"\seberlein\powfacpy\powfacpy_tests") # You may change the project path.
# Make sure the path starts with "\"

# Simple Parameter Study
We use the class `PFStudyCases`. We want to create study case with various values for the active and reactive power of a load. 

First we set the attribute `parameter_values` and define a dictionary with parameter names and a list of values. Each element in the lists is used in one study case. 

In [ ]:
pfsc = powfacpy.PFStudyCases(app)
pfsc.parameter_values = {
    "p HV load":[ 1,  2, 1, 2],
    "q HV load":[-1, -1, 1, 1],
}

Here we have defined four study cases and in the first case `p HV load` equals 1 and `q HV load` equals -1. The parameter names must not contain *?=",\~|()! as these characters cannot be used for names of objects in PF.

Next we  connect the variables to the PF database by defining a dictionary with paths in the attribute `parameter_paths` using the same parameter names as keys. The paths include the attribute name (e.g. `plini` which is the active power of the load). 

In [ ]:
pfsc.parameter_paths = {
    "p HV load":r"Network Model\Network Data\Grid\General Load HV\plini",
    "q HV load":r"Network Model\Network Data\Grid\General Load HV\qlini",
}

Finally we define the active grids. For now, we assume that for all cases exactly one grid is active (we look at more complicated cases later). 

In [ ]:
pfsc.active_grids = r"Network Model\Network Data\test_case_studies\Grid 1"

Now let's create the study cases.

In [ ]:
pfsc.create_cases()

This creates the following study cases:

![cases_1](cases_1.png)

The names of the study cases are created according to the parameter names and their values ("_" is used instead of "=" which is not allowed in PF object names). Furthermore, the corresponding operation scenarios are created: 

![scenarios_1](scenarios_1.png)

For more complicated case studies with further parameters, it is advisable to structure the cases in folders. We define the attribute `hierarchy` using a list of parameter names (in this case with only one element) that corresponds to folder directories.

In [ ]:
pfsc.hierarchy = ["q HV load"]
pfsc.create_cases()

Now `q HV load` is used for the folder names and not in the study case name.

![cases_2](cases_2.png)

The structure of the operation scenarios is similar.

# Advanced Parameter Study
Let's look at a bit more complicated parameter study.

In [ ]:
pfsc = powfacpy.PFStudyCases(app)
pfsc.parameter_values = {
    "p HV load":[1, 2, 1, 2, 1, 2, 1, 2,],
    "q HV load":[-1, -1, 1, 1,-1, -1, 1, 1,],
    "control": ["A","A","A","A","B","B","B","B",]
}
pfsc.parameter_paths = {
    "p HV load":r"Network Model\Network Data\Grid\General Load HV\plini",
    "q HV load":r"Network Model\Network Data\Grid\General Load HV\qlini",
}
pfsc.active_grids = r"Network Model\Network Data\test_case_studies\Grid 2"

An additional parameter `control` is defined. Note that there is no parameter path defined for this parameter. It can be seen as a placeholder for the study cases structure and to do more complicated things than simply setting an attribute of a PF object later (we'll see below).

By default, study cases and variants are created. If variations should be created in addition, use:

In [ ]:
pfsc.add_variation_to_each_case = True

This can be necessary because some parameters, like attributes of DSL objects, are stored in variants and not in operation scenarios.
Let's again create the study cases.

In [ ]:
pfsc.hierarchy = ["control","q HV load"]
pfsc.parent_folder_variations = r"Network Model\Variations"
pfsc.create_cases()

This creates the following structure in the network variations (as well as in the scenarios and study cases):

![variations_3](variations_3.png)

Let's set further parameters, add plots (using plot interface) and simulate (using dyn. simulation interface) for each study case. The study case objects are stored in the `study_cases` attribute. We can iterate through the cases and set parameters depending on the previously defined `control` parameter.



In [ ]:
pfpi = powfacpy.PFPlotInterface(app)
pfds = powfacpy.PFDynSimInterface(app)

for case_num,study_case_obj in enumerate(pfsc.study_cases):
    study_case_obj.Activate()
    # Set controller parameters
    dsl_controller_obj = r"Network Model\Network Data\test_case_studies\Grid 2\WECC WT Control System Type 4A\REEC_A Electrical Control Model"
    if pfsc.get_value_of_parameter_for_case("control",case_num) == "A":
        pfsc.set_attr(dsl_controller_obj,{"PfFlag":0,"VFlag":1}) 
    elif pfsc.get_value_of_parameter_for_case("control",case_num) == "B":
        pfsc.set_attr(dsl_controller_obj,{"PfFlag":1,"VFlag":0}) 
    # Plot
    pfpi.clear_plot_pages()
    pfpi.set_active_plot("Reactive current","WPP")
    pfpi.plot(dsl_controller_obj,"s:Iqcmd")
    # Simulate   
    pfds.initialize_and_run_sim() 

Let's walk through this piece of code. The plotting and simulation interfaces are instantiated to be used later. 

It is iterated through the study cases objects (and case numbers) and each study case is activated. Then the controller parameters are set. The value of the `control` parameter is obtained using the method `get_value_of_parameter_for_case` and the controller parameters are set depending on the controller type (`A` or `B`). This provides the flexibility to adjust the study cases depending on previously defined parameters in any way (e.g. set parameters, activate grids,...). And all settings are stored in the previously defined operation scenarios/variations.

Finally, a plot is created for every case and the dynamic simulation is executed.  

# Compare Study Cases
It is often required to compare the results of cases. In the following, we use the previously defined cases to compare the simulation results for cases where the parameter `p HV load` equals 1 (note that you may have to "Rebuild" the plot in the upper left for PF to show it correctly):  

In [ ]:
# Create a study case where results from other cases are compared in a plot
study_case_comparison = pfsc.create_by_path(r"Study Cases\Comparison.IntCase")
study_case_comparison.Activate()
pfpi.clear_plot_pages()
pfpi.set_active_plot("Reactive current controller setpoint for p HV load = 1","Comparison ")
for case_num,study_case_obj in enumerate(pfsc.study_cases):  
    dsl_controller_obj = r"Network Model\Network Data\test_case_studies\Grid 2\WECC WT Control System Type 4A\REEC_A Electrical Control Model"
    if pfsc.get_value_of_parameter_for_case("p HV load",case_num) == 1:
        # Get results object from respective study case 
        results_obj = pfsc.get_single_obj(
            "All calculations",parent_folder=study_case_obj)
        # Create label for plot
        case_label = pfsc.get_case_params_value_string(case_num,
            omitted_parameters="p HV load",
            delimiter=" | ",
            equals_symbol="=") 
        label = "Iq setpoint (" + case_label + ")"
        # Plot
        pfpi.plot(dsl_controller_obj,"s:Iqcmd",
            results_obj=results_obj,
            label=label)
        pfpi.autoscale()    

Let's walk through the code. After creating a study case where the results are compared in a plot, it is again iterated through all cases. For cases where `p HV load` has a certain value, the result object is fetched. Next we want to create a label for the curve in the plot. We use the method `get_case_params_value_string` with certain options for the delimiter and the equals symbol (here we have more options than for PF object names, i.e. `=` and `|` are allowed). Moreover, the parameter `p HV load` is omitted because it is the same for all compared cases (i.e. 1, see condition). Further information on the variable is added to the label and the curve is finally added to the plot.

An alternative to iterating through all study cases is to get certain cases using the method `get_study_cases`:

In [ ]:
study_cases_p_HV_load_equals_1 = pfsc.get_study_cases({"p HV load":lambda x: x==1})
for study_case_obj in study_cases_p_HV_load_equals_1:
    # do something with these specific cases

`get_study_cases` takes a dictionary as input, where lambda functions with conditions for parameters are defined. Note also that the methods `get_value_of_parameter_for_case` and `get_case_params_value_string` also work with study case objects as inputs instead of case numbers.